In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import StratifiedKFold

from utilities import load_data

In [2]:
X,y = load_data(90, amplitude=10)

In [3]:
X.shape #2n samples of 400x400 pixels, with a depth of 1 (greyscale type)

(180, 400, 400, 1)

In [4]:
'''
conv2D(32, (3,3), activation='tanh') 
conv2D(32, (3,3), activation='tanh') 
conv2D(32, (3,3), activation='tanh') 
conv2D(32, (3,3), activation='tanh') 
Flatten()
Dense(units=256, activation='relu')
Dense(units=1, activation='sigmoid')

'''

"\nconv2D(32, (3,3), activation='tanh') \nconv2D(32, (3,3), activation='tanh') \nconv2D(32, (3,3), activation='tanh') \nconv2D(32, (3,3), activation='tanh') \nFlatten()\nDense(units=256, activation='relu')\nDense(units=1, activation='sigmoid')\n\n"

In [5]:
skfold = StratifiedKFold(n_splits=4)

acc_per_fold = []
loss_per_fold = []

for train, test in skfold.split(X,y):

    #define the model
    model = keras.models.Sequential()
    
    model.add(keras.layers.Conv2D(32, (3, 3), activation='tanh', input_shape=X.shape[1:]))
    model.add(keras.layers.AveragePooling2D((3, 3)))
    
    model.add(keras.layers.Conv2D(32, (3, 3), activation='tanh'))
    model.add(keras.layers.AveragePooling2D((2, 2)))

    model.add(keras.layers.Conv2D(32, (3, 3), activation='tanh'))
    model.add(keras.layers.AveragePooling2D((2, 2)))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    if len(acc_per_fold)==0 : model.summary()
    #compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

    #fit the model
    history = model.fit(X[train], y[train], epochs=10, batch_size=25, verbose=1)

    #evaluate the model
    print('\nEvaluation of the model : \n')
    scores = model.evaluate(X[test], y[test])
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    print("\n-----------------------------------\n")


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 398, 398, 32)      320       
_________________________________________________________________
average_pooling2d (AveragePo (None, 132, 132, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 130, 130, 32)      9248      
_________________________________________________________________
average_pooling2d_1 (Average (None, 65, 65, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 32)        9248      
_________________________________________________________________
average_pooling2d_2 (Average (None, 31, 31, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 30752)             0

In [6]:
acc_per_fold

[100.0, 100.0, 100.0, 100.0]